In [3]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score

In [5]:
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\decision tree\data.xlsx')
df1=df

In [6]:
# Drop rows containing 'fail' or 'not satisfied' in any column
#df1 = df1.drop(df1[(df1 == 'fail').any(axis=1) | (df1 == 'not satisfied').any(axis=1)].index)
dup_df = df.copy()
new_df = pd.concat([df, dup_df], axis=0)
new_df.reset_index(drop=True, inplace=True)

In [7]:
new_df.shape

(202, 16)

In [8]:
X = new_df.drop(columns=['Eligible', 'Queue', 'Application status ', 'Workflow status'])
y = new_df[['Eligible', 'Queue', 'Application status ', 'Workflow status']]

In [9]:
X = pd.get_dummies(X)

In [53]:
# Take last 15 records for test, rest for train
X_train, X_test = X.iloc[:-20], X.iloc[-20:]
y_train, y_test = y.iloc[:-20], y.iloc[-20:]

# Initialize and train the multi-output decision tree classifier
tree_model = MultiOutputClassifier(DecisionTreeClassifier(random_state=42))
tree_model.fit(X_train, y_train)

# Make predictions
y_pred = tree_model.predict(X_test)

In [54]:
from sklearn.model_selection import cross_val_score

accuracy_scores = {}

for target in y_train.columns:
    y_train_target = y_train[[target]]
    
    # Perform cross-validation on the current target variable
    scores = cross_val_score(tree_model, X_train, y_train_target, cv=10, scoring='accuracy')
    
    # Compute and store accuracy score for the current target variable
    accuracy_scores[target] = scores.mean()

# Print accuracy scores for each target variable
for target, score in accuracy_scores.items():
    print(f"Accuracy for {target}: {score}")

C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(


Accuracy for Eligible: 0.9722222222222221
Accuracy for Queue: 1.0
Accuracy for Application status : 0.9833333333333332
Accuracy for Workflow status: 1.0


In [12]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Initialize an empty dictionary to store confusion matrices for each target variable
confusion_matrices = {}

# Compute confusion matrix for each target variable
for target in y_test.columns:
    y_test_target = y_test[target]  # Extract the target variable from test data
    y_pred_target = y_pred[:, y_test.columns.get_loc(target)]  # Extract corresponding predictions
    
    # Compute confusion matrix
    cm = confusion_matrix(y_test_target, y_pred_target)
    
    # Add confusion matrix to the dictionary
    confusion_matrices[target] = cm

# Print confusion matrices for each target variable
for target, cm in confusion_matrices.items():
    print(f"Confusion matrix for {target}:")
    print(cm)
    print()

Confusion matrix for Eligible:
[[ 8  0  0]
 [ 0 13  0]
 [ 2  0  2]]

Confusion matrix for Queue:
[[13  0]
 [ 0 12]]

Confusion matrix for Application status :
[[ 4  0  0]
 [ 0 13  0]
 [ 2  0  6]]

Confusion matrix for Workflow status:
[[13  0]
 [ 0 12]]



In [13]:
new_data = pd.DataFrame({
    'new/renewals': ['new'],
    'Grade': ['other grade'],
    'Document found': ['no'],
    'Electronic': ['no'],
    'military, foster child, out of home care, Law enforcement ': ['na'],
    'Direct certication': ['no'],
    'Owner/operator': ['no'],
    'DOB verified': ['na'],
    'Month /prior month document date ': ['na'],
    'Primary Address/name matched ': ['na'],
    'Secondary address/name  match if any': ['na'],
    'Total income available': ['na']
})

In [14]:
def check_failure(new_data):
    for column in new_data.columns:
        if 'fail' in new_data[column].values or 'not satisfied' in new_data[column].values:
            return True
    return False

In [15]:
if check_failure(new_data):
    print("Output: under review")
else:

    # Apply one-hot encoding to the new data
    new_data_encoded = pd.get_dummies(new_data)

    # Ensure new data has the same columns as X_train
    new_data_encoded = new_data_encoded.reindex(columns=X_train.columns, fill_value=0)

    # Make predictions on new data
    new_predictions = tree_model.predict(new_data_encoded)

    # Decode predictions
    new_predictions_decoded = pd.DataFrame(columns=y.columns, data=new_predictions)

    print(new_predictions_decoded)

  Eligible    Queue Application status  Workflow status
0  pending  On hold             On hold         on hold


In [16]:
import joblib

# Save the entire MultiOutputClassifier model using joblib
joblib.dump(tree_model, 'decision_tree_model.pkl')

['decision_tree_model.pkl']

In [55]:
dt_classifier = MultiOutputClassifier(DecisionTreeClassifier(class_weight='balanced'))

dt_classifier.fit(X_train, y_train)

y_pred = dt_classifier.predict(X_test)

In [56]:
accuracy_scores = {}

for target in y_train.columns:
    y_train_target = y_train[[target]]
    
    # Perform cross-validation on the current target variable
    scores = cross_val_score(dt_classifier, X_train, y_train_target, cv=4, scoring='accuracy')
    
    # Compute and store accuracy score for the current target variable
    accuracy_scores[target] = scores.mean()

# Print accuracy scores for each target variable
for target, score in accuracy_scores.items():
    print(f"Accuracy for {target}: {score}")

Accuracy for Eligible: 0.9835748792270531
Accuracy for Queue: 1.0
Accuracy for Application status : 0.9835748792270531
Accuracy for Workflow status: 1.0


In [57]:
print("Unique values in y_train:")
for column_name, column_values in y_train.items():
    unique_values = column_values.unique()
    print(f"Column '{column_name}': {unique_values}")

print("Unique values in y_test:")
for column_name, column_values in y_test.items():
    unique_values = column_values.unique()
    print(f"Column '{column_name}': {unique_values}")

Unique values in y_train:
Column 'Eligible': ['pending' 'eligible both' 'Eligible' 'FTC ineligible ' 'na' 'On hold']
Column 'Queue': ['Manual' 'na' 'On hold']
Column 'Application status ': ['Sensitive note' 'Sensitive note/close' 'On hold' 'on hold']
Column 'Workflow status': ['Received na under review' 'Review completed ' 'on hold']
Unique values in y_test:
Column 'Eligible': ['na' 'pending' 'On hold']
Column 'Queue': ['Manual' 'On hold']
Column 'Application status ': ['Sensitive note' 'On hold' 'on hold']
Column 'Workflow status': ['Received na under review' 'on hold']
